<a href="https://colab.research.google.com/github/Sohampatel26/GOD-AI/blob/main/GOD_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n


In [ ]:
from datasets import concatenate_datasets

In [ ]:
final_dataset= load_dataset("nikita200/geeta")

In [ ]:
print(final_dataset)

In [ ]:
# Split the dataset into training and test sets (e.g., 80% train, 20% test)
train_test_split = final_dataset['train'].train_test_split(test_size=0.3, seed=42)

# Further split the test set into validation and test sets (e.g., 50% validation, 50% test)
test_valid_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

# Combine the splits into a single dataset dictionary
final_dataset = {
    'train': train_test_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test']
}

NameError: name 'dataset_split' is not defined

In [ ]:
final_dataset['train'][0]

{'instruction': '5. What is the significance of the battlefield in the Bhagavad Gita?',
 'output': "The battlefield in the Bhagavad Gita symbolizes the inner struggle of the human soul. The war between the Pandavas and Kauravas represents the eternal battle between good and evil, and the battlefield is the setting where the ultimate truth and wisdom are revealed to Arjuna by Krishna. It also represents the human mind and the constant struggle between our desires and our duty. The Gita teaches that true victory can only be achieved by conquering one's inner enemies and finding inner peace and harmony. The battlefield also serves as a reminder that life itself is a battle and one must face challenges with courage and determination.",
 'input': ''}

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )


In [ ]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [ ]:
%%time
from transformers import set_seed, GPT2Tokenizer, GPT2LMHeadModel

# Set seed for reproducibility
seed = 42
set_seed(seed)

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def gen(model, prompt, max_length):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
    return [tokenizer.decode(outputs[0], skip_special_tokens=True)]

# Select an index to test
index = 10

# Retrieve the instruction and output from the dataset
prompt = final_dataset['test'][index]['instruction']
summary = final_dataset['test'][index]['output']

# Format the prompt for the model
formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"

# Generate the output using the model
res = gen(model, formatted_prompt, 100)

# Process the output
output = res[0].split('Output:\n')[1] if 'Output:\n' in res[0] else res[0]

# Display the results
dash_line = '-' * 100
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Give the best fit response for this question."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['instruction']}" if sample["instruction"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [ ]:
from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, final_dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    final_dataset = final_dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    final_dataset = final_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['instruction', 'output','input'],
    )

    # Filter out samples that have input_ids exceeding max_length
    final_dataset = final_dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    final_dataset = final_dataset.shuffle(seed=seed)

    return final_dataset

In [ ]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, final_dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, final_dataset['validation'])


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [ ]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=100,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_trainer.train()

In [ ]:
from transformers import BartConfig
import torch

# Define quantization configuration
qconfig = torch.quantization.get_default_qconfig('qnnpack')

# Define model configuration
bnb_config = BartConfig(
    quantization_enabled=True,
    model_type='bart',
    static=[True],
    quant_types=['weight'],
    int8=[False],
    config_file=None
)


In [ ]:
!pip install accelerate

In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)